# Exploration et test de modèles de prédiction du prix de l'habitation sur le dataset des Demandes de Valeurs Foncières sur le département du Seine et Marne

## 1. Téléchargement des données

La section suivante permet de télécharger les données depuis le site data.gouv.fr. Les données sont téléchargées dans le dossier `data/`

Elles sont disponibles à l'adresse suivante : <https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/>

In [1]:
import os
import requests
from tqdm.notebook import tqdm

def download_data():
    if "data" not in os.listdir():
        os.mkdir("data")

    # Fetch urls
    r = requests.get(
        "https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/")
    urls = set([url.split('"')[1] for url in r.text.split()
                if "valeursfoncieres-" in url and "txt" in url])

    for idx, url in enumerate(tqdm(urls)):
        local_filename = url.split('/')[-1].split('.')[0] + ".txt"
        if local_filename not in os.listdir("data"):
            print(f"Downloading {url}")
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(f"data/{url.split('/')[-1].split('.')[0]}.txt", 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
download_data()

  0%|          | 0/6 [00:00<?, ?it/s]

## 2. Chargement des données

In [2]:
import pandas as pd

dvfDf = pd.concat([pd.read_csv(
    f"data/{file}", sep="|", low_memory=False) for file in tqdm(os.listdir("data"))])
dvfDf = dvfDf.dropna(axis=1, thresh=0.5*len(dvfDf.index))
dvfDf

  0%|          | 0/7 [00:00<?, ?it/s]

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,Code commune,Section,No plan,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3627125,1.0,30/12/2019,Vente,"1400000,00",24.0,RUE,8752,SAINT SAUVEUR,75002.0,PARIS 02,...,102.0,AM,18.0,3.0,4.0,Local industriel. commercial ou assimilé,100.0,0.0,NaN,NaN
3627126,1.0,17/12/2019,Adjudication,"620000,00",14.0,RUE,7195,PAVEE,75004.0,PARIS 04,...,104.0,AM,14.0,2.0,2.0,Appartement,45.0,2.0,NaN,NaN
3627127,1.0,05/12/2019,Vente,"370000,00",23.0,RUE,7561,POISSONNIERE,75002.0,PARIS 02,...,102.0,AH,67.0,2.0,4.0,Local industriel. commercial ou assimilé,47.0,0.0,NaN,NaN
3627128,1.0,12/12/2019,Adjudication,"44000,00",2.0,RUE,6840,NOTRE DAME DES VICTOIRES,75002.0,PARIS 02,...,102.0,AJ,127.0,1.0,3.0,Dépendance,0.0,0.0,NaN,NaN


## Preprocessing

In [3]:
dvfDf["Date mutation"] = pd.to_datetime(
    dvfDf["Date mutation"], format="%d/%m/%Y")
dvfDf = dvfDf.sort_values(by="Date mutation")

In [4]:
dvfDf["Code postal"] = pd.to_numeric(
    dvfDf["Code postal"], errors="coerce").astype("Int64")
dvfDf["Type local"] = dvfDf["Type local"].astype("category").cat.add_categories(
    "Terrain").fillna("Terrain").astype(str)
dvfDf["Nombre pieces principales"] = pd.to_numeric(
    dvfDf["Nombre pieces principales"], errors="coerce").astype("Int64").fillna(0)
dvfDf["Valeur fonciere"] = pd.to_numeric(
    dvfDf["Valeur fonciere"].str.replace(",", "."), errors="coerce")

In [5]:
dvfDf.count()

No disposition               19765458
Date mutation                19765458
Nature mutation              19765458
Valeur fonciere              19569317
No voie                      12249314
Type de voie                 11839817
Code voie                    19599887
Voie                         19599312
Code postal                  19599070
Commune                      19765458
Code departement             19765458
Code commune                 19765458
Section                      19764759
No plan                      19765458
Nombre de lots               19765458
Code type local              11441930
Type local                   20146827
Surface reelle bati          11428987
Nombre pieces principales    20146827
Nature culture               13430274
Surface terrain              13430274
dtype: int64

In [6]:
seine_et_marneDf = dvfDf[dvfDf["Code departement"] == "77"].copy(deep=True)
del dvfDf
seine_et_marneDf = seine_et_marneDf.dropna(
    subset=["Valeur fonciere"])
seine_et_marneDf["new_commune"] = ("77" +
                                   seine_et_marneDf["Code commune"].astype(str).str.zfill(3)).astype("Int64")
seine_et_marneDf["new_commune"]

ValueError: invalid literal for int() with base 10: '77243.0'

In [ ]:
seine_et_marneDf["Surface Totale"] = seine_et_marneDf.loc[:, seine_et_marneDf.columns.str.contains(
    "Surface")].applymap(lambda x: float(x.replace(",", ".")) if type(x) != float else x).sum(axis=1, skipna=True)
seine_et_marneDf

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,No plan,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,new_commune,Surface Totale
1373477,1,2018-07-02,Vente,100000.0,92.0,RUE,1610,SAINT DENIS,77400,LAGNY-SUR-MARNE,...,417,1,2.0,Appartement,22.0,1,NaN,NaN,77243,22.0
1360974,1,2018-07-02,Vente,262000.0,NaN,NaN,B025,SAINT GERMAIN,77860,SAINT-GERMAIN-SUR-MORIN,...,133,0,NaN,Terrain,NaN,0,S,8.0,77413,8.0
1360973,1,2018-07-02,Vente,262000.0,5.0,RUE,0520,LOUIS DOUVIZIE,77860,SAINT-GERMAIN-SUR-MORIN,...,132,0,1.0,Maison,84.0,3,S,110.0,77413,194.0
1360972,1,2018-07-02,Vente,262000.0,NaN,NaN,B025,SAINT GERMAIN,77860,SAINT-GERMAIN-SUR-MORIN,...,131,0,NaN,Terrain,NaN,0,S,297.0,77413,297.0
1360971,1,2018-07-02,Vente,262000.0,NaN,NaN,B025,SAINT GERMAIN,77860,SAINT-GERMAIN-SUR-MORIN,...,130,0,NaN,Terrain,NaN,0,S,8.0,77413,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162200,1,2023-06-30,Vente,12500.0,28.0,RUE,1610,SAINT DENIS,77400,LAGNY-SUR-MARNE,...,145,1,3.0,Dépendance,0.0,0,NaN,NaN,77243,0.0
1162201,1,2023-06-30,Vente,790889.0,245.0,AV,0128,DE LA GRANDE HAIE,77100,MAREUIL-LES-MEAUX,...,520,0,4.0,Local industriel. commercial ou assimilé,1663.0,0,S,2630.0,77276,4293.0
1162202,1,2023-06-30,Vente,790889.0,245.0,AV,0128,DE LA GRANDE HAIE,77100,MAREUIL-LES-MEAUX,...,520,0,4.0,Local industriel. commercial ou assimilé,0.0,0,S,2630.0,77276,2630.0
1162203,1,2023-06-30,Vente,790889.0,NaN,AV,0128,DE LA GRANDE HAIE,77100,MAREUIL-LES-MEAUX,...,523,0,NaN,Terrain,NaN,0,S,1363.0,77276,1363.0


In [13]:
seine_et_marneDf.to_csv("data/seine_et_marne.csv", index=False)

## Création du modèle

In [18]:
# modeling the evolution of the price of property with a sklearn using new_commune, Type local, Surface Totale, Nombre pieces principales, and Date mutation
from sklearn.model_selection import train_test_split

numeric_features = ["Surface Totale", "Nombre pieces principales"]
categorical_features = ["new_commune", "Type local"]
date_features = ["Date mutation"]

def preparation_data(df, numeric_features, categorical_features, date_features, test_size=0.2):
    X = df[numeric_features + categorical_features + date_features]
    y = df["Valeur fonciere"]

    return train_test_split(X, y, test_size=test_size, random_state=42)

X_train, X_test, y_train, y_test = preparation_data(seine_et_marneDf, numeric_features, categorical_features, date_features)

,new_commune,Type local,Surface Totale,Nombre pieces principales,Date mutation
3513434,77288,Terrain,0.0,0,2022-03-25
3651933,77349,Terrain,554.0,0,2021-12-27
2701911,77350,Terrain,8.0,0,2020-12-21
2784594,77464,Maison,593.0,5,2019-06-21
2757994,77371,Terrain,1164.0,0,2019-07-11
...,...,...,...,...,...
2690361,77458,Terrain,773.0,0,2020-09-04
3618630,77108,Appartement,62.0,3,2021-01-22
1375901,77181,Dépendance,0.0,0,2018-08-27
1137316,77288,Appartement,90.0,3,2023-03-22


In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FunctionTransformer, Pipeline
from sklearn.preprocessing import RobustScaler

def creation_model(numeric_features, categorical_features, date_features):
    numeric_transformer = Pipeline(steps=[
        ('scaler', RobustScaler())])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])


    def date_convert_to_timestamp(X):
        return X.values.astype(int) // 10 ** 9


    date_transformer = FunctionTransformer(date_convert_to_timestamp)

    preprocessor = ColumnTransformer(
        transformers=[
                                    ('num', numeric_transformer, numeric_features),
                                    ('cat', categorical_transformer,
                                    categorical_features),
                                    ('date', date_transformer, date_features)], remainder="passthrough")

    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', RandomForestRegressor(n_jobs=-1, random_state=42))], verbose=True)

    model.fit(X_train, y_train)

    return model

model = creation_model(numeric_features, categorical_features, date_features)

y_pred = model.predict(X_test)
model.score(X_test, y_test)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.5s
[Pipeline] ......... (step 2 of 2) Processing regressor, total= 8.9min


0.9188754763963021

In [32]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, model.predict(X_test), squared=False)

1324465249446.7327

In [33]:
import pickle

pickle.dump(model, open("long_training_model.pkl", "wb"))

In [24]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FunctionTransformer, Pipeline
from sklearn.preprocessing import RobustScaler

numeric_features = ["Surface Totale", "Nombre pieces principales"]
categorical_features = ["new_commune", "Type local"]
date_features = ["Date mutation"]


numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


def date_convert_to_timestamp(X):
    return X.values.astype(int) // 10 ** 9


def preprocessed_todense(X):
    return X.toarray()


date_transformer = FunctionTransformer(date_convert_to_timestamp)

tonumpy_transformer = FunctionTransformer(preprocessed_todense)

preprocessor = ColumnTransformer(
    transformers=[
                                ('num', numeric_transformer, numeric_features),
                                ('cat', categorical_transformer,
                                 categorical_features),
                                ('date', date_transformer, date_features)], remainder="passthrough")

best_hgb_model = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('to_numpy', tonumpy_transformer),
                                 ('regressor', HistGradientBoostingRegressor(categorical_features=[2, 3], random_state=42, learning_rate=0.1, max_depth=5, max_iter=200, max_leaf_nodes=15, min_samples_leaf=9))], verbose=True)
best_hgb_model.fit(X_train, y_train)
best_hgb_model.score(X_test, y_test)

[Pipeline] ...... (step 1 of 3) Processing preprocessor, total=   0.3s
[Pipeline] .......... (step 2 of 3) Processing to_numpy, total=   0.3s
[Pipeline] ......... (step 3 of 3) Processing regressor, total=  28.1s


0.8051892548100184

In [27]:
print("HGB RMSE: ", mean_squared_error(
    y_test, best_hgb_model.predict(X_test), squared=False))
print("RF  RMSE: ", mean_squared_error(y_test, model.predict(X_test), squared=False))

HGB MSE:  3180543327236.147
RF  MSE:  1324465249446.7327


In [28]:
import plotly.express as px
# Make a prediction for the next year for a 100m2 house with 4 rooms in 77078 and plot the evolution of the price of property

fh = pd.date_range(start="2023-01-01", end="2024-01-01", freq="M")

forecast = model.predict(
    pd.DataFrame({"new_commune": [77078]*len(fh), "Type local": ["Maison"]*len(fh), "Surface Totale": [100]*len(fh), "Nombre pieces principales": [4]*len(fh), "Date mutation": fh}))
forecast = pd.Series(forecast, index=fh)
px.line(forecast, title="Evolution of the price of property in 77078 for a 100m2 house with 4 rooms")